# CONTINUOUS MODEL VERSION - PARASOL CELLS

This is a full simulation of the continuous model version for parasol cells. The simulation time resolution is 0.2ms.

In [ ]:
# -*- coding: utf-8 -*-
import fileinput
import os.path
from scipy.optimize import curve_fit
from scipy import integrate

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import itertools

pgf_with_rc_fonts = {"font.family": "serif","font.serif": [],"font.sans-serif": []}
plt.rcParams.update(pgf_with_rc_fonts)

plt.close('all')

First, define all the needed filter functions. $t_{on}$, $t_{off}$ and `fac` $\equiv p$ are the main parameters for this simulation. They are left as default values here.

In [ ]:
#plot numerically integrated results of moving edge
def temporalFilter(t,tau1,tau2,p):
    return (t*t*t/(tau1*tau1*tau1*tau1)*np.exp(-t/tau1)-p*t*t*t/(tau2*tau2*tau2*tau2)*np.exp(-t/tau2))
	
def spatialFilter(x,x0,sigma,alpha,beta):
    return np.exp(-(x-x0)*(x-x0)/(2*sigma*sigma))-alpha*np.exp(-(x-x0)*(x-x0)/(2*beta*beta*sigma*sigma))
def spatialFilterCenter(x,x0,sigma):
    return np.exp(-(x-x0)*(x-x0)/(2*sigma*sigma))
def spatialFilterSurround(x,x0,sigma,alpha,beta):
    return alpha*np.exp(-(x-x0)*(x-x0)/(2*beta*beta*sigma*sigma))

t_ons = [10.] #[5.,2.5,7.5,13.,5.,5.,2.5,10.]
t_offs = [15.] #[15.,15.,15.,15.,15.,15.,7.5,30.]
fac = [0.8] #[0.8,0.8,0.8,0.8,0.65,0.95,0.8,0.8] #p 

For each of these triples, define the standard values for the filters.

In [ ]:
for to in zip(t_ons,t_offs,fac):
    #just to make sure that the variables have the right values	
    on_tau1 = to[0]
    on_tau2 = to[1]
    on_p = to[2]

    alpha = .1
    beta = 3. #CSR
    sigma = .5 # parasolic/midget cell ratio * sigma (=1)
    px_midget_ratio = 1. #pixel to receptor ratio, needed for receptor distance/number
    px_dist= 0.5 #in arcmin, from paper
    par_m_ratio = 4. #2arcmin center field sigma from paper
    x0 = 0
    
    p_min_threshold = 55. #threshold such that linear motion with vel=1. doesn't cause reation in parasol cells

    tfv = []
    tfo = []
    spat = []
    me=[]
    me2=[]

This method is used for the simulation of a bar moving linearly in one direction, with different on/off periods. The spatial filters are applied first, as they are not dependent on the on/off periods (for off periods, the spatial filter process will lead 0 as a result for all cells).

In [ ]:
    #this is the simulated time, as the cycle interval is 105ms 
    st = np.linspace(0,430,430*5)
    st = np.linspace(0,210,210*5)

    for i in range(430*5):
        me+=[0.]
        me2+=[0.]
        
    #the velocity of the moving bar in units of the simulation
    vel=1. #2., 0.5, 0.25, 4.
    velocity=0.625*120.*0.001*vel 
    velocity_stat=0.625*120.*0.001 #=0.075

    #get the spatially filtered signals
    for step in range(0,int((500/vel+2*630)*5)):
        spat += [integrate.quad(spatialFilter,-630*velocity-30*vel+0.2*velocity*step,-630*velocity+0.2*velocity*step,args=(x0,sigma,alpha,beta))[0]] #0.1*step
    
    #generate the bare temporal filter curve
    for step in range(0,5*200):	
        tfv += [temporalFilter(0.2*step,on_tau1,on_tau2,on_p)] #0.1*step,on_tau1,on_tau2,on_p)]	

    psf = []
    for x in range(65):
        xi=x-32
        psf += [spatialFilter(xi,0,8.*sigma,alpha,beta)]
    psflen2 = len(psf)/2
    
    spat=np.asarray(spat)

In [ ]:
    fig_labels = ["a","b","c","d","e","f","g","h"]

    #if |f'(t)| > epsilon -> take data for area
    epsilon = 0.0005
    meps = 20.
    mdthr = 15.
    mdthr2 = 0.1
    #motion detector time interval as related to the velocity
    mdf = 1. 

    #open files
    dfile = 'jitter/parasol/jcad_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))+'.txt'
    treg = 'jitter/parasol/data/tdiff_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))
    me_r_treg = 'jitter/parasol/data/me_r_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))
    me105_r_treg = 'jitter/parasol/data/me105_r_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))


Start the iteration for the different periods.

In [ ]:
    for period in [60,105,210]:
        fig, (ax) = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(5,6))

        if period == 60:
            onpers = [60,15,30,45] 
            stretch = 105./60.
        if period == 105:
            onpers = [105,15,30,45,60,75,90] 
            stretch = 1.
        if period == 210:
            onpers = [210,30,60,90,120,150,180]
            stretch = 0.5

        cdata_file = open(dfile,'a+')
        cdata_file.write(str(period)+'\t'+str(0)+'\t'+str(0.)+'\t'+str(0.)+'\t'+str(0.)+'\n')
        cdata_file.close()

Iterate through different op-duty periods.

In [ ]:
        j=0
        for onp in onpers:
            print onp, period
            #INITIALIZE
            #if that's the case, get the baseline me105_l/r
            if onp == period:
                me105_l=[]
                me105_r=[]
                for i in range(430*5):
                    me105_l+=[0.]
                    me105_r+=[0.]
                me105_l_2=[]
                me105_r_2=[]
                for i in range(430*5):
                    me105_l_2+=[0.]
                    me105_r_2+=[0.]
            else:
                me_l=[]
                me_r=[]
                for i in range(430*5):
                    me_l+=[0.]
                    me_r+=[0.]
                me_l_2=[]
                me_r_2=[]
                for i in range(430*5):
                    me_l_2+=[0.]
                    me_r_2+=[0.]

In [ ]:
            #spatial filter input is just a moving heaviside convolved with the filter function
    
            #used during process
            star=np.zeros((int(float(len(spat)-5.*630.)/5.*velocity_stat),630*5))
            xmnl = int(float(len(spat)-5.*630.)/5.*velocity_stat)
            #for each midget cell in simulated area
            for xn in range(xmnl):
                xz=xn
                kl=0
                tfo =[]
                sfo =[]
                sfo_step =[]

                #for each timestep
                for step in range(0,630*5):
                    
                    #create off period for specific midget cell
                    if onp != period:  #otherwise it will alternate for one step in that case
                        
                        #SET VALUES TO 0, IF OFF DUTY PERIOD
                        #onp: on period, vary here for 52.5 and 210ms (0.2*stretch*step)
                        if (0.2*stretch*step)%(105) >= stretch*onp: 
                            sfo += [0.]
                        else:
                            sfo += [spat[int(float(xn)/velocity_stat)*5 +step]]
                            
                    else:
                        sfo += [spat[int(float(xn)/velocity_stat)*5 +step]]

After setting the spatially filtered values to zero for off-duty times, the temporal filter can be apllied to the adopted input.

In [ ]:
                #temporal filtering 
                for step in range(420*5):
                    sfo_step.insert(0, sfo[step])
                    if step > 200*5:
                        del sfo_step[-1]

                    tfvs=[]
                    #convolution
                    step_ts = sum(itertools.imap(lambda x,y: x*y, tfv, sfo_step))
                    tfo = step_ts 
                    #rectification
                    if tfo>0 :
                        star[xz][step]= tfo

After the filter stages for the midget cells/subunits, the rectified signal can be spatially filtered gain by the parasol cells.

In [ ]:
            #parasol part
            pstar=np.zeros((int(float(len(spat)-5.*630.)/5.*velocity_stat/8.), 630*5))#
            xpnl = int(float(len(spat)-5.*630.)/5.*velocity_stat/8.)
            for xn in range(xpnl):
                #distance between two parasol cells
                xz=8*xn
                kl=0
                
                #here circular bdry cond.
                for step in range(0,420*5):
                    for xsm in range(xz-32,xz+33):
                        #parasol filter value times subunit value at time step
                        pstar[xn][step] += psf[xsm-xz+psflen2]*star[(xsm+xmnl)%xmnl][step]
                    #parasol threshold
                    pstar[xn][step] -= p_min_threshold
                    #rectification
                    if pstar[xn][step] <= 0.:
                        pstar[xn][step] = 0.

Now the motion detection mechanism is implemented. For details, cf. thesis.

In [ ]:
            thr = 0.
            mdf = 1.
        
            #new motion detection mechanism
            #motion energy is the current value plus the last one
            meps = 0.
            for xn in range(xpnl):
                xz=8*xn
                for step in range(0,420*5):
                    #avoid initialization effects
                    if step > 200*5:
                        if onp != period:
                            if (pstar[xn][step] > pstar[xn][step-int(mdf*5./velocity)]) and pstar[xn][step] > meps:
                                if pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] > pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me_r[step-200*5] += pstar[xn][step] + pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                                elif pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] < pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me_r[step-200*5] -= pstar[xn][step] + pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                            elif (pstar[xn][step] < pstar[xn][step-int(mdf*5./velocity)]) and pstar[xn][step] > meps:
                                if pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] < pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me_r[step-200*5] += pstar[xn][step] + pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                                elif pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] > pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me_r[step-200*5] -= pstar[xn][step] + pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                        #if baseline
                        else:
                            if (pstar[xn][step] > pstar[xn][step-int(mdf*5./velocity)]) and pstar[xn][step] > meps:
                                if pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] > pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me105_r[step-200*5] += pstar[xn][step] + pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                                elif pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] < pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me105_r[step-200*5] -= pstar[xn][step] + pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                            elif (pstar[xn][step] < pstar[xn][step-int(mdf*5./velocity)]) and pstar[xn][step] > meps:
                                if pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] < pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me105_r[step-200*5] += pstar[xn][step] + pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)] 
                                elif pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] > pstar[(xn-1+xpnl)%xpnl][step-int(mdf*5./velocity)]:
                                    me105_r[step-200*5] -= pstar[xn][step] + pstar[(xn+1+xpnl)%xpnl][step-int(mdf*5./velocity)] 

Evaluation part. Save all curves to files. 

`tdiff` is the differece of motion signal between the current on-duty time and the baseline, meaning 100% on-duty period.

In [ ]:
            if onp != period:
                me105_l = np.asarray(me105_l)
                me105_r = np.asarray(me105_r)

                me_r = np.asarray(me_r)
                me_l = np.asarray(me_l)

                me105_l_2 = np.asarray(me105_l_2)
                me105_r_2 = np.asarray(me105_r_2)

                me_r_2 = np.asarray(me_r_2)
                me_l_2 = np.asarray(me_l_2)
                
                #veldiff_l = me_l - me_l_2
                #veldiff_r = me_r - me_r_2

                #here comes the actual output part
                diff = me_r-me_l
                diff105 = me105_r-me105_l

                diff_2 = me_r_2-me_l_2
                diff105_2 = me105_r_2-me105_l_2
                
                #if contrast should be taken into account
                #contr = float(onp)/float(period)*np.ones(len(diff105))
                
                tdiff = diff-diff105 
                
                #save curves to files
                t_data = open(treg+'_per'+str(period)+'_onp'+str(onp)+'.npy','w+')
                np.savetxt(t_data, tdiff.astype(float), delimiter=",")
                t_data.close()
                diff_data = open(treg+'_per'+str(period)+'_onp'+str(onp)+'.npy','w+')
                np.savetxt(diff_data, diff.astype(float), delimiter=",")
                diff_data.close()
                diff_105_data = open(treg+'_per'+str(period)+'_onp'+str(onp)+'.npy','w+')
                np.savetxt(diff_105_data, diff105.astype(float), delimiter=",")
                diff_105_data.close()
                me_r_data = open(me_r_treg+'_per'+str(period)+'_onp'+str(onp)+'.npy','w+')
                np.savetxt(me_r_data, me_r.astype(float), delimiter=",")
                me_r_data.close()
                me105_r_data = open(me105_r_treg+'_per'+str(period)+'_onp'+str(onp)+'.npy','w+')
                np.savetxt(me105_r_data, me_r.astype(float), delimiter=",")
                me105_r_data.close()

Calculating the output $S(t_{on})$. The area between the curves is integrated.
The total motion signal $S_{tot}$ has to be larger than the threshold $0<\varepsilon$. Also the signal itself has to be larger than 0.

In [ ]:
                thr = min(tdiff)

                contr = float(onp)/float(period)

                dx = 0.
                dy = 0.
                area = 0.
                
                df = [tdiff[step+1]-tdiff[step] for step in range(len(me105_l)-2)]
                
                for step in range(len(me105_l)-2):
                    if tdiff[step] < 0. and me_r[step]*me_r[step] > 0:  #if |f'| > epsilon
                        dx += 0.2   # step width, use boxes
                        dy += np.sqrt((tdiff[step]+tdiff[step+1])*(tdiff[step]+tdiff[step+1]))/2.  # |displacement| -> mean value

                area = dx*dy

                cdata_file = open(dfile,'a+')
                cdata_file.write(str(period)+'\t'+str(onp)+'\t'+str(area)+'\t'+str(dx)+'\t'+str(dy)+'\n')
                cdata_file.close()

                df += [df[len(me105_l)-3]]
                df += [df[len(me105_l)-3]]

                df = np.asarray(df)

Some more lines to plot the resulting curves for `tdiff` etc.

In [ ]:
                ax[j/2][j%2].set_title("("+fig_labels[j]+") on " + str(onp)+" ms", fontsize=11)
                ax[j/2][j%2].plot(st[0:210*5], me_l[10*5:220*5], color='limegreen', label='leftward')
                ax[j/2][j%2].plot(st[0:210*5], me_r[10*5:220*5], color='darkgreen', label='rightward')
                ax[j/2][j%2].plot(st[0:210*5], me105_l[10*5:220*5], color='orange', label='leftward')
                ax[j/2][j%2].plot(st[0:210*5], me105_r[10*5:220*5], color='red', label='rightward')
                ax[j/2][j%2].plot(st[0:210*5], diff[10*5:220*5], color='black', label='diff')

                if period == 210:
                    ax[j/2][j%2].axvspan(210.-(105.-stretch*onp), 210., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(420.-(105.-stretch*onp), 420., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                if period == 105:
                    ax[j/2][j%2].axvspan(105.-(105.-stretch*onp), 105., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(210.-(105.-stretch*onp), 210., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(315.-(105.-stretch*onp), 315., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(420.-(105.-stretch*onp), 420., color='lightgray', linewidth=0, alpha=0.5,zorder=2)

                if period == 60:
                    ax[j/2][j%2].axvspan(60.-(60.-stretch*onp), 60., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(120.-(60.-stretch*onp), 120., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(180.-(60.-stretch*onp), 180., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(240.-(60.-stretch*onp), 240., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(200.-(60.-stretch*onp), 200., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(360.-(60.-stretch*onp), 360., color='lightgray', linewidth=0, alpha=0.5,zorder=2)
                    ax[j/2][j%2].axvspan(420.-(60.-stretch*onp), 420., color='lightgray', linewidth=0, alpha=0.5,zorder=2)

                tdd = tdiff[10*5:220*5]
                y2 = np.ma.masked_less(me_r, 0.01)

                ax[j/2][j%2].set_xlim([0.,210.]) #420.])
                ax[j/2][j%2].set_xticks(np.arange(0, 211, 52.5)) #[0.,105.,210.,315.,420.])
                ax[j/2][j%2].tick_params(labelsize=8)

                if onp == 15:
                    ax[j/2][j%2].legend(fontsize=8,ncol=2,loc='upper left')

                j+=1

Plot the curves for each time period to one file.

In [ ]:
        #add the values for the whole on-duty period to the list (all equal 0.)
        cdata_file = open(dfile,'a+')
        cdata_file.write(str(period)+'\t'+str(period)+'\t'+str(0.)+'\t'+str(0.)+'\t'+str(0.)+'\n')
        cdata_file.close()

        fig.text(0.015, 0.5, '$S(t)$ [Arbitrary Units]', ha='center', va='center', fontsize=8, rotation=90)
        fig.text(0.5,0.01, 'Time $t$ [ms]', ha='center', fontsize=8)
        plt.tight_layout()
        
        plt.show()
        plt.savefig('jitter/parasol/jmsa_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))+'_'+str(period)+'ms.pdf')
        plt.savefig('jitter/parasol/jmsa_par_nm_55_on'+str(int(on_tau1))+'_off'+str(int(on_tau2))+'_p'+str(int(100*on_p))+'_'+str(period)+'ms.pgf')

        plt.close('all')